In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import re
import time

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
!which chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


# Scraping NASA Mars News

In [3]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [4]:
time.sleep(2)
news_html = browser.html
news_soup = bs(news_html, 'html.parser')

In [5]:
titles = news_soup.find_all("div", class_="content_title")
articles = news_soup.find_all("div", class_="article_teaser_body")

In [6]:
news_title = titles[1].text
news_p = articles[0].text

In [7]:
results_dic = {"news_title":news_title, "news_p":news_p}

# JPL Mars Space Images - Featured Image

In [8]:
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

In [9]:
image_html = browser.html
image_soup = bs(image_html,'html.parser')

In [10]:
image_result = image_soup.find("a", class_="button fancybox")

In [11]:
featured_image_url = "https://www.jpl.nasa.gov" + image_result["data-fancybox-href"]

In [12]:
results_dic["featured_image_url"] = featured_image_url

In [13]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19673_ip.jpg'

# Mars Weather

In [14]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)
time.sleep(3)

In [15]:
weather_html = browser.html
weather_soup = bs(weather_html,'lxml')

In [16]:
pattern = re.compile(r'InSight sol')
mars_weather = weather_soup.find('span', text=pattern)
print(mars_weather)

None


In [17]:
results_dic["mars_weather"] = mars_weather

# Mars Facts

In [18]:
facts_url = "https://space-facts.com/mars/"

In [19]:
tables = pd.read_html(facts_url)
fact_df = tables[0]

In [20]:
fact_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [21]:
html_table = fact_df.to_html(index = False, header=False)

results_dic["html_table"] = html_table

In [22]:
html_table

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemispheres

In [23]:
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)

In [24]:
hemi_list = ["Cerberus", "Schiaparelli", "Syrtis Major", "Valles Marineris"]

In [25]:
hemisphere_image_urls = []

for hemi in hemi_list:
    
    browser.click_link_by_partial_text(hemi)
    
    hemi_html = browser.html
    hemi_soup = bs(hemi_html,'html.parser')
   
    img_url = hemi_soup.find_all("li")[0].a["href"]
    title = hemi + " Hemisphere"
    
    hemisphere_image_urls.append({"title":title, "img_url":img_url})

In [26]:
results_dic["hemi_img_urls"] = hemisphere_image_urls

In [27]:
results_dic

{'news_title': "NASA's Perseverance Rover Mission Getting in Shape for Launch",
 'news_p': 'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. ',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19673_ip.jpg',
 'mars_weather': None,
 'html_table': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</t